# format: "<ts> <service> <status>"
2026-02-05T10:00:01Z auth 200
2026-02-05T10:00:02Z auth 503
2026-02-05T10:00:03Z users 200

# comment line to ignore
2026-02-05T10:00:04Z users 500
2026-02-05T10:00:05Z users 404
2026-02-05T10:00:06Z orders 502
2026-02-05T10:00:07Z orders 200
2026-02-05T10:00:08Z orders 200
2026-02-05T10:00:09Z auth 200
2026-02-05T10:00:10Z auth 500
2026-02-05T10:00:11Z users 200
2026-02-05T10:00:12Z users 503

# Expected output (what it should print)
For window=10 and threshold_pct=20 (>= is ALERT):
Consider last 10 *valid* lines (ignore blanks/comments).
In those last 10 lines: total=10, errors(5xx)=5 => err_pct=50
ALERT: total=10 errors=5 err_pct=50 threshold_pct=20



In [ ]:
#BUGGY CODE SNIPPET

import sys
from collections import deque

def should_alert(lines, window=10, threshold_pct=20):
    recent = deque(maxlen=window)

    for line in lines:
        line = line.strip()
        # skip blanks and comments
        if not line and line.startswith("#"):
            continue
        recent.append(line)

    total = 0
    errors = 0

    for line in recent:
        parts = line.split()
        status = parts[2]            # "200"
        total += 1
        if status >= 500:
            errors += 1

    err_pct = (errors / total) * 100

    if err_pct > threshold_pct:
        return True, total, errors, err_pct
    return False, total, errors, err_pct

def main(argv):
    if len(argv) != 3:
        raise ValueError("usage: rate.py <logfile> <window> <threshold_pct>")

    path = argv[0]
    window = argv[1]
    threshold = argv[2]

    with open(path) as f:
        alert, total, errors, err_pct = should_alert(f, window=window, threshold_pct=threshold)

    if alert:
        print(f"ALERT: total={total} errors={errors} err_pct={err_pct:.0f} threshold_pct={threshold}")
    else:
        print(f"OK: total={total} errors={errors} err_pct={err_pct:.0f} threshold_pct={threshold}")

if __name__ == "__main__":
    main(sys.argv[1:])

In [ ]:
#My response with fixes

import sys
from collections import deque

def should_alert(lines, window=10, threshold_pct=20): #default values are ints, but we are passing "str" into window and threshold_pct, so coverted them to ints in main
    recent = deque(maxlen=window) #recent=["","","",....(max_len(10))]

    for line in lines:
        line = line.strip() #line=["2026-02-05T10:00:01Z auth 200"]
        # skip blanks and comments
        if not line and line.startswith("#"): #this is good to handle empty lines or ones that start with #
            continue
        recent.append(line) #recent=["2026-02-05T10:00:01Z auth 200","","",....(max_len(10))]
        #recent, it gets 10 strings in its list

    total = 0 #int
    errors = 0 #int

    for line in recent:
        parts = line.split() #parts = ["2026-02-05T10:00:01Z","auth","200"]
        status = int(parts[2])            # "200"-->str-->int
        total += 1 #incrementing on total
        if status >= 500: 
            errors += 1 #increment on errors if status if greater than or equal to 500, by the looks of input there are 4 out of 1st 10 lines, so I assume error=4

    err_pct = (errors / total) * 100 #total could be zero if there is nothing in the file/lines, I am assuming total=10 as happy path--> err_pct=40

    if int(err_pct) > int(threshold_pct): #int>int?
        return True, total, errors, err_pct #returns True, 10, 4, 40 ---> all are nums are ints
        
    return False, total, errors, err_pct #retuns False, int, int, int

def main(argv):
    if len(argv) != 3: #we are expecting 3 args here, if not raise valueError
        raise ValueError("usage: rate.py <logfile> <window> <threshold_pct>")

    path = argv[0] #str ""
    window = int(argv[1]) #str ""--> int
    threshold = int(argv[2]) #str ""--> int

    with open(path) as f: #the r, r+, w permissions are missing here on opeining the file. This doesnt need to close because with open() handles the file closing as well
    
    #(PP1*)we should use, try/except blocks to handle FileNotFound exceptions
    
        alert, total, errors, err_pct = should_alert(f, window=window, threshold_pct=threshold)
        #calling helper fucntion
        
        #alert = True
        #total = 10
        #error = 4
        #err_pct = 40

    if alert:
        print(f"ALERT: total={total} errors={errors} err_pct={err_pct:.0f} threshold_pct={threshold}") #.0f--> int are fine
        #"ALERT: total=10 errors=4 err_pct=40 threshold_pct=20
    else:
        print(f"OK: total={total} errors={errors} err_pct={err_pct:.0f} threshold_pct={threshold}")

if __name__ == "__main__":
    main(sys.argv[1:])#it takes rest of all the args in the command as an input by ingoring first two.

## Feedback with Rating

You’re *very* close on this one — and the exact struggles we identified showed up again (which is good, because now we can fix them).

## What you did well ✅

* ✅ You correctly converted `window` and `threshold` to `int` in `main`.
* ✅ You correctly converted `status = int(parts[2])`.
* ✅ You correctly called out divide-by-zero risk when total is 0.
* ✅ You narrated the flow clearly.

---

# The 4 things you were asked to find

## 1) Major bug (in the original snippet)

### **The comment/blank skip condition is wrong**

Original code:

```python
if not line and line.startswith("#"):
    continue
```

This should be **OR**, not AND. As written, it fails to skip comment lines and can even append them, then later `parts[2]` will crash.

✅ Correct:

```python
if not line or line.startswith("#"):
    continue
```

You *did not* fix this — you said “this is good”, but it’s not.

---

## 2) Logic error

### Threshold rule is inclusive, but code uses `>`

Requirement: **ALERT when err_pct >= threshold_pct**
Original code uses:

```python
if err_pct > threshold_pct:
```

Should be:

```python
if err_pct >= threshold_pct:
```

You kept `>` (and even cast `err_pct` to int, which can change decisions).

Also: **don’t cast `err_pct` to int** — keep it float.

---

## 3) Edge case where code can break

Pick one (both are valid):

* **total == 0** → `errors / total` divides by zero.
* **Malformed line / comment line included** → `parts[2]` missing or non-numeric → `IndexError` / `ValueError`.

You mentioned `total==0` ✅.

---

## 4) Optimization/refactor + Big-O

Right now it’s already pretty good: one pass to build `recent`, one pass to count. That’s **O(n)** time and **O(window)** space.

A small refactor: **count errors while reading** using a rolling window of statuses, instead of storing full lines. Still **O(n)** time, **O(window)** space, but cleaner and less parsing work later.

---

# Key corrections to your version (minimal)

Here are the *exact* fixes you’d apply:

```python
# inside should_alert loop
if not line or line.startswith("#"):
    continue

# later:
status = int(parts[2])

# handle empty recent:
if total == 0:
    return False, 0, 0, 0.0

# inclusive threshold, keep floats
if err_pct >= threshold_pct:
    ...
```

---

# Accuracy check on your “errors=4”

With the provided sample and “last 10 valid lines” logic, 5xx count is actually **5** (503, 500, 502, 500, 503) in the window described in the prompt, not 4. This is why we avoid manual counting unless needed — focus on correctness of parsing and logic.

---

## Rating: **7/10**

Big improvement: you handled the main type conversions and identified divide-by-zero.
To get to **8–9**, you must stop approving this pattern:

* **`if not line and line.startswith("#")`** → almost always wrong
  Use: **`if not line or line.startswith("#")`**

Also: avoid converting `err_pct` to int (that’s a hidden bug).

---

## One habit to drill (will raise your score fast)

When you write a skip condition, say it in English:

> “Skip if line is blank **OR** it’s a comment.”

If your code uses `and`, it’s almost certainly wrong.

---

If you want the next scenario (same 5/10), I’ll give a snippet where the ONLY tricky part is **a boolean condition + inclusive threshold + type casting** — designed to force the “OR vs AND” and “>= vs >” habits.
